In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import itertools
import matplotlib

In [2]:
import os
os.makedirs('plots/TC', exist_ok=True)
os.makedirs('plots/MSC', exist_ok=True)
os.makedirs('processed/TC', exist_ok=True)
os.makedirs('processed/MSC', exist_ok=True)

In [3]:
#sheet = 'Multi Session Chat View 1'
# sheet = 'Topical Chat View 1'
# df = pd.read_excel('ITL_20230521.xlsx', sheet_name=sheet)
# df = df.loc[df.Model != "Gap"]

# Remove unnamed columns
# df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# New Results from data/evals.csv
df = pd.read_csv('data/evals.csv')
# dataset_prefix = "MSC"
dataset_prefix = "TC"
df = df.loc[df.dataset == dataset_prefix]

In [4]:
if dataset_prefix == "MSC":
    # title_fontsize = 36
    legend_fontsize = 18
    label_fontsize = 24
else:
    # dataset_prefix = "TC"
    # title_fontsize = 
    legend_fontsize = 20
    label_fontsize = 20

    # Rename "Knowledge" as "Persona Signal"
    df = df.rename(columns={"Knowledge": "Persona Signal"})

# Set to a common linux font
fonts = {'family' : 'serif',
        'serif' : 'Caladea',
        # 'weight' : 'bold',
        'size'   : 36
        }
# For some wierd reason you need to set the 
# font size like following
# 36 for MSC
# 40 for TC 

matplotlib.rc('font', **fonts)

In [5]:
df['Prompt Type'].unique()

array(['Short', 'Orig', 'Short (ppl)', 'Orig (ppl)'], dtype=object)

In [6]:
df['Prompt Type'].replace("Orig(.*)", "FS-ICL\\1", regex=True, inplace=True)
df['Prompt Type'].replace("Short(.*)", "ZS-ICL\\1", regex=True, inplace=True)

In [7]:
# Drop for values ppl1, ppl2, Exp1, Exp2. Or if it is null
df = df.loc[~df['Prompt Type'].isin(['ppl1', 'ppl2', 'Exp1', 'Exp2'])]
df = df.loc[~df['Prompt Type'].isnull()]

In [8]:
df['Prompt Type'].unique()

array(['ZS-ICL', 'FS-ICL', 'ZS-ICL (ppl)', 'FS-ICL (ppl)'], dtype=object)

In [9]:
df["History Signal"].unique()

array(['Full', 'PegasusFT', 'BART', 'Recent-2', 'Recent-4', 'Recent-8',
       'Recent-10', 'Semantic-2', 'Semantic-4', 'Semantic-8',
       'Semantic-10', 'None', 'Pegasus-CD', 'Recent-1', 'Semantic-1'],
      dtype=object)

In [10]:
# MSC Rules
df['History Signal'].replace(r"Prev (\d+)", r"Recent-\1", regex=True, inplace=True)
df['History Signal'].replace("4 Semant.*", "Semantic-4", regex=True, inplace=True)
df['History Signal'].replace(r"(\d+) semantic sim \(simcse\)", r"Semantic-\1", regex=True, inplace=True)
df['History Signal'].replace("semantic sim (simcse)", "Semantic-4", inplace=True)
df['History Signal'].replace("Pegasus cnn/dm", "Pegasus-CD", inplace=True)
df['Persona Signal'].replace("Pegasus cnn/dm", "Pegasus-CD", inplace=True)
# # TC Conversion Rules
df['History Signal'].replace("4 semantic sim (simcse)", "Semantic-4", inplace=True)
df['History Signal'].replace("4 semantic sim", "Semantic-4", regex=True, inplace=True)
df['Model'].replace("T-k instruct", "Tk-Instruct", inplace=True)
df['Model'].replace("Tk-instruct", "Tk-Instruct", inplace=True)

In [11]:
df["History Signal"].unique()

array(['Full', 'PegasusFT', 'BART', 'Recent-2', 'Recent-4', 'Recent-8',
       'Recent-10', 'Semantic-2', 'Semantic-4', 'Semantic-8',
       'Semantic-10', 'None', 'Pegasus-CD', 'Recent-1', 'Semantic-1'],
      dtype=object)

In [12]:
df['Model'].replace("text-davinci-003", "GPT-3", inplace=True)


In [13]:
# Drop rows with Persona Signal = PegasusFT
df = df.loc[df['Persona Signal'] != "PegasusFT"]

In [14]:
df.sample(15)

,dataset,Model,batch_size,prompt_type,few_shot,background_knowledge,History Signal,history_k,wandb,num_gpus,...,Persona Signal,Prompt Type,Method,BLEU,rouge1,rouge2,rougeL,Bert-p,Bert-r,Bert-f1
248,TC,T0,12,manual,True,False,Semantic-10,10,False,1,...,None,FS-ICL,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
261,TC,T0,12,ppl,False,False,Semantic-8,8,False,1,...,None,ZS-ICL (ppl),0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
294,TC,Tk-Instruct,12,manual,True,False,Full,4,False,1,...,None,FS-ICL,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
258,TC,T0,12,ppl,False,False,Recent-10,10,False,1,...,None,ZS-ICL (ppl),0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
474,TC,GPT-3,12,manual,True,False,Recent-1,1,False,1,...,None,FS-ICL,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
282,TC,Tk-Instruct,12,manual,False,False,BART,4,False,1,...,None,ZS-ICL,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
462,TC,GPT-3,12,manual,True,True,PegasusFT,4,False,8,...,Pegasus-CD,FS-ICL,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
440,TC,GPT-3,12,manual,False,False,Recent-4,4,False,8,...,None,ZS-ICL,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
242,TC,T0,12,manual,True,False,Recent-4,4,False,1,...,None,FS-ICL,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
260,TC,T0,12,ppl,False,False,Semantic-4,4,False,1,...,None,ZS-ICL (ppl),0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [15]:
used_cols = ['Model', 'Method', 'Persona Signal', 'History Signal', 'Prompt Type',
       'BLEU', 'METEOR', 'rouge1', 'rouge2', 'rougeL', 'Bert-p', 'Bert-r',
       'Bert-f1', 'DEB', 'Bleurt', 'output', 'prompt']
df = df[used_cols]
df['total_budget'] = df['output'] + df['prompt']
df['BLEURT']=df['Bleurt']

In [16]:
df = df.sort_values(['Model', 'Prompt Type'])

In [17]:
df

,Model,Method,Persona Signal,History Signal,Prompt Type,BLEU,METEOR,rouge1,rouge2,rougeL,Bert-p,Bert-r,Bert-f1,DEB,Bleurt,output,prompt,total_budget,BLEURT
450,GPT-3,0.5,None,Full,FS-ICL,0.5,0.156904,0.5,0.5,0.5,0.5,0.5,0.5,0.952048,0.363777,39.621147,798.293515,837.914662,0.363777
451,GPT-3,0.5,None,PegasusFT,FS-ICL,0.5,0.134215,0.5,0.5,0.5,0.5,0.5,0.5,0.824261,0.336097,37.871058,271.986549,309.857607,0.336097
452,GPT-3,0.5,None,BART,FS-ICL,0.5,0.134018,0.5,0.5,0.5,0.5,0.5,0.5,0.822749,0.336770,37.702655,270.801265,308.503919,0.336770
453,GPT-3,0.5,None,Recent-2,FS-ICL,0.5,0.141298,0.5,0.5,0.5,0.5,0.5,0.5,0.895496,0.340845,37.341573,322.257394,359.598967,0.340845
454,GPT-3,0.5,None,Recent-4,FS-ICL,0.5,0.144644,0.5,0.5,0.5,0.5,0.5,0.5,0.904315,0.342487,38.920052,410.100036,449.020087,0.342487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,flanT5-XL,0.5,Pegasus-CD,PegasusFT,ZS-ICL (ppl),0.5,0.114752,0.5,0.5,0.5,0.5,0.5,0.5,0.792150,0.300725,48.384473,351.893535,400.278007,0.300725
209,flanT5-XL,0.5,Pegasus-CD,BART,ZS-ICL (ppl),0.5,0.114752,0.5,0.5,0.5,0.5,0.5,0.5,0.792150,0.300725,48.384473,351.893535,400.278007,0.300725
378,flanT5-XL,0.5,None,Pegasus-CD,ZS-ICL (ppl),0.5,0.122815,0.5,0.5,0.5,0.5,0.5,0.5,0.841282,0.360340,40.130991,141.813335,181.944326,0.360340
379,flanT5-XL,0.5,None,Recent-1,ZS-ICL (ppl),0.5,0.129332,0.5,0.5,0.5,0.5,0.5,0.5,0.783195,0.354326,37.009398,109.671032,146.680429,0.354326


In [18]:
# # set and increase Matplotlib font
# plt.rcParams['font.family'] = 'sans-serif'
# plt.rcParams['font.sans-serif'] = 'Arial'
# plt.rcParams['font.size'] = 24

# font = {'family' : 'normal',
#         'weight' : 'bold',
#         'size'   : 32}

# matplotlib.rc('font', **font)

In [19]:
def drop_nan_row_cols(df):
    # print("Shape of df: {}".format(df.shape))
    # Print the columns/rows that will be dropped
    # print("Columns with all nan: {}".format(df.columns[df.isnull().all(axis=0)]))
    # print("Rows with all nan: {}".format(df.index[df.isnull().all(axis=1)]))
    # Drop rows with all nan
    df = df.dropna(axis=0, how='all')
    # Drop columns with all nan
    df = df.dropna(axis=1, how='all')
    # print("Shape of df after dropping rows and columns with all nan: {}".format(df.shape))
    return df

def scatter_plot_pivoted_df(pivoted_df, metric, save_path, is_roi=False):
    # Print shape
    # print(pivoted_df)

    print("Metric: {}".format(metric))
    print("Save path: {}".format(save_path))
    
    # Check if any row or column is all nan
    # if pivoted_df.isnull().all(axis=1).any() or pivoted_df.isnull().all(axis=0).any():
    #     print("Skipping plot because of all nan row or column for {} @ {}".format(metric, save_path))
    #     return

    # # Drop rows with all nan
    # pivoted_df = pivoted_df.dropna(axis=0, how='all')
    # # Drop columns with all nan
    # pivoted_df = pivoted_df.dropna(axis=1, how='all')


    if pivoted_df.shape[0] == 0:
        print("Skipping plot for {} @ {}".format(metric, save_path))
        return

    colors = sns.color_palette('Paired', len(pivoted_df.columns))
    markers = ['o', 'v', 's', 'p', 'P', '*', 'X', 'D', 'd', '1', '2', '3', '4', '8', 'h', 'H', 'x', 'X', 'D', 'd', '|', '_']

    # Create scatter plot
    fig, ax = plt.subplots(figsize=(8,6))
    # print(pivoted_df.index.tolist())
    # Scatter plot
    models = pivoted_df.columns
    for i, model in enumerate(models):
        x = pivoted_df.index.tolist()
        y = pivoted_df[model]
        # y shouldn't be all nans
        if y.isnull().all():
            continue
        lbl = f"{model[0]} ({model[1].replace('-ICL', '')})"
        ax.scatter(x, y, color = colors[i], marker = markers[i], label=lbl, s=150)

    # Set x-axis labels
    ax.set_xticks(list(range(len(pivoted_df.index.tolist()))))
    ax.set_xticklabels(pivoted_df.index.tolist(), rotation=90, fontsize=label_fontsize)

    # Set y-axis tick font
    ax.tick_params(axis='y', labelsize=label_fontsize)

    # Add legend at the top of the figure
    # show in a single row
    # Smaller font size
    if len(models) <= 5:
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.26), ncol=2, fontsize=legend_fontsize)
    else:
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.43), ncol=2, fontsize=legend_fontsize)

    ax.grid(True, axis='y')
    # x-axis label
    ax.set_xlabel('History Signal', fontsize=label_fontsize)
    # y-axis label
    if is_roi:
        ax.set_ylabel(f'ROI ({metric})', fontsize=label_fontsize)
    else:
        ax.set_ylabel(f'{metric}', fontsize=label_fontsize)


    # plt.tight_layout()
    # Save such that legends are fitted within the figure
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()

    return

    # Separate plot averaged over all models (two groups FS and ZS)
    fig, ax = plt.subplots(figsize=(8,6))

    # Scatter plot
    models = pivoted_df.columns
    fs_models = [model for model in models if "FS" in model[1]]
    x = pivoted_df.index.tolist()
    y = pivoted_df[fs_models].mean(axis=1)
    ax.scatter(x, y, color = colors[0], marker = markers[0], label='Model Avg. (FS)', s=150)


    zs_models = [model for model in models if "ZS" in model[1]]
    y = pivoted_df[zs_models].mean(axis=1)
    ax.scatter(x, y, color = colors[1], marker = markers[1], label='Model Avg. (ZS)', s=150)

    # Set x-axis labels
    ax.set_xticks(list(range(len(pivoted_df.index.tolist()))))
    ax.set_xticklabels(pivoted_df.index.tolist(), rotation=90, fontsize=label_fontsize)

    # Set y-axis tick font
    ax.tick_params(axis='y', labelsize=label_fontsize)
    
    # Add legend at the top of the figure
    # show in a single row
    # Smaller font size
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.16), ncol=2, fontsize=legend_fontsize)

    ax.grid(True, axis='y')
    # x-axis label
    ax.set_xlabel('History Signal', fontsize=label_fontsize)
    # y-axis label
    if is_roi:
        ax.set_ylabel(f'ROI ({metric})', fontsize=label_fontsize)
    else:
        ax.set_ylabel(f'{metric}', fontsize=label_fontsize)

    
    # plt.tight_layout()
    # Save such that legends are fitted within the figure
    plt.savefig(save_path.replace('.pdf', '_avg.pdf'), bbox_inches='tight', dpi=300)
    plt.close()

In [20]:
def calculate_ROI(df_perf, df_budget, metric_exp=10.0):
    # print(df_perf)
    assert df_perf.shape == df_budget.shape
    # Find the rows with index "None_0" or "No DH, no BI", only one of these should exist
    if "None_0" in df_perf.index:
        X_perf = df_perf.loc['None_0']
        X_budget = df_budget.loc['None_0']
    # elif "No DH, no BI" in df_perf.index:
    #     assert "None_0" not in df_perf.index
    #     X_perf = df_perf.loc['No DH, no BI']
    #     X_budget = df_budget.loc['No DH, no BI']
    else:
        raise ValueError("None_0 not found in df_perf.index.")

    # assert df_perf.index[0] == "None" or df_perf.index[0] == "No DH, no BI"
    # Match index and column order
    assert df_perf.index.tolist() == df_budget.index.tolist()
    assert df_perf.columns.tolist() == df_budget.columns.tolist()

    # Calculate ROI: w.r.t to best baseline in "None"
    # df_delta = df_perf - X_perf
    # df_delta_budget = df_budget - X_budget
    # Unnormalized ROI
    df_delta = (100*df_perf)**metric_exp
    df_delta_budget = df_budget
    df_roi = 100*df_delta / df_delta_budget
    return df_roi.iloc[1:]

In [21]:
def proc_sub_df_persona(sub_df, baseline_df, signal, prompt_type_groups, metric, all_subframes, dataset_prefix):
    for prompt_types_name, prompt_types in prompt_type_groups.items():
        # prompt_sub_df = sub_df
        prompt_sub_df = sub_df[sub_df['Prompt Type'].isin(prompt_types)]
        # Remove old baselines
        # prompt_sub_df = sub_df[~(sub_df['History Signal'] == "None")]
        # Rename this baseline to "None_1"
        prompt_sub_df.loc[prompt_sub_df['History Signal'] == 'None', 'History Signal'] = 'None_1'
        
        # Add common baseline - prompt type would be non-ppl
        # prompt_baseline_df = baseline_df[baseline_df['Prompt Type'].isin(prompt_types)]
        prompt_baseline_df = baseline_df[baseline_df['Prompt Type'].isin(prompt_type_groups['normal'])]
        # overwrite prompt type of baseline
        for i, row in prompt_baseline_df.iterrows():
            if row['Prompt Type'] == 'FS-ICL':
                prompt_baseline_df.at[i, 'Prompt Type'] = prompt_types[0]
            elif row['Prompt Type'] == 'ZS-ICL':
                prompt_baseline_df.at[i, 'Prompt Type'] = prompt_types[1]
        # rename this baseline to "None_0"
        prompt_baseline_df['History Signal'] = "None_0"
        prompt_sub_df  = prompt_sub_df.append(prompt_baseline_df)
        
        print("History:", prompt_sub_df['History Signal'].unique())
        print("Prompt types:", prompt_sub_df['Prompt Type'].unique())

        # Remove blenderbot
        prompt_sub_df = prompt_sub_df[~prompt_sub_df['Model'].isin(["BlenderBot-3B"])]
        if len(prompt_sub_df) <= 0:
            continue

        pivoted_df = prompt_sub_df.pivot(index='History Signal', columns=['Model', 'Prompt Type'], values=metric)        
        pivoted_budget = prompt_sub_df.pivot(index='History Signal', columns=['Model', 'Prompt Type'], values='total_budget')
        pivoted_input_budget = prompt_sub_df.pivot(index='History Signal', columns=['Model', 'Prompt Type'], values='prompt')

        # Fixed model order
        history_order = ["None_0", "None_1", "BART", "Pegasus-CD", "PegasusFT", 
                         "Recent-1", "Recent-2", "Recent-4", "Recent-8", "Recent-10", "Recent-16", 
                         "Semantic-1", "Semantic-2", "Semantic-4", "Semantic-8", "Semantic-10", "Semantic-16","Full"
                         ]
        # remove history signals that are not in the data
        history_order = [h for h in history_order if h in prompt_sub_df['History Signal'].unique()]
        pivoted_df = pivoted_df.reindex(history_order)
        pivoted_budget = pivoted_budget.reindex(history_order)
        pivoted_input_budget = pivoted_input_budget.reindex(history_order)

        # ROI: based on first row (None)
        ROI = calculate_ROI(pivoted_df, pivoted_budget)

        models = ['flanT5-XL',
                'T0',
                'Tk-Instruct',
                'GPT-3']
        # remove models that are not in the data
        models = [m for m in models if m in prompt_sub_df['Model'].unique()] 
        # icls = ["ZS-ICL", "FS-ICL", "ZS-ICL (ppl)", "FS-ICL (ppl)"]
        icls = prompt_types
        model_icl_pairs = list(itertools.product(models, icls))
        pivoted_df = pivoted_df.reindex(model_icl_pairs, axis=1)
        pivoted_budget = pivoted_budget.reindex(model_icl_pairs, axis=1)
        pivoted_input_budget = pivoted_input_budget.reindex(model_icl_pairs, axis=1)
        ROI = ROI.reindex(model_icl_pairs, axis=1)

        # Clean the dataframes of empty columns/rows
        pivoted_df = drop_nan_row_cols(pivoted_df)
        pivoted_budget = drop_nan_row_cols(pivoted_budget)
        pivoted_input_budget = drop_nan_row_cols(pivoted_input_budget)
        ROI = drop_nan_row_cols(ROI)

        all_subframes[signal + "_" + prompt_types_name] = pivoted_df
        all_subframes[signal + "_" + prompt_types_name + "_budget"] = pivoted_budget
        all_subframes[signal + "_" + prompt_types_name + "_ibudget"] = pivoted_input_budget
        if "None_0" in pivoted_df.index:
            all_subframes[signal + "_" + prompt_types_name + "_ROI"] = ROI
        else:
            print("Cannot calculate ROI without NoHistory baseline (i.e. None).")

        # Plot absolute metrics
        f1 = f'plots/{dataset_prefix}/{metric}_{signal}_{prompt_types_name}.pdf'
        scatter_plot_pivoted_df(pivoted_df, metric, f1)

        # Plot ROI
        if "None" in pivoted_df.index:
            f2 = f'plots/{dataset_prefix}/{metric}_{signal}_{prompt_types_name}_ROI.pdf'
            scatter_plot_pivoted_df(ROI, metric, f2, is_roi=True)

In [22]:
def merge_runs(framedict, metric, dataset_prefix, target="ppl"):
    assert target in ["normal", "ppl"], "merge target must be either normal or ppl"

    # DF Merged, Only for None_ppl, PegasusFT_*
    def is_budget(x):
        return x.endswith("_budget") or x.endswith("_ibudget")
    
    def is_roi(x):
        return x.endswith("_ROI")
    
    def is_runlist(x):
        # Set of runs that we want to keep
        # No persona and PegasusFT-based persona summary
        return x.startswith("None") or x.startswith("PegasusFT") or x.startswith("Pegasus-CD")

    modded_run_keys = list(
        filter(
            lambda x: is_runlist(x) and not is_roi(x) and not is_budget(x),
            framedict.keys()
        )
    )
    df_modded_runs_group = {k: v for k, v in framedict.items() if k in modded_run_keys}

    budget_keys = list(
        filter(lambda x: is_budget(x), framedict.keys())
    )
    df_budgets = {k.replace("_budget", ""): v for k, v in framedict.items() if k in budget_keys}
    df_budgets = {k: v for k, v in df_budgets.items() if k in df_modded_runs_group}

    # Baseline is None_normal, index "None"
    df_baseline = df_modded_runs_group["None_normal"].iloc[:1]
    df_baseline_budget = df_budgets["None_normal"].iloc[:1]
    # check that history is None
    assert df_baseline.index[0] == "None_0" and df_baseline_budget.index[0] == "None_0", "Baseline must be None_0"

    to_remove = []
    if target == "ppl":
        for k, v in df_modded_runs_group.items():
            if k.endswith("_normal"):
                to_remove.append(k)
    elif target == "normal":
        for k, v in df_modded_runs_group.items():
            if k.endswith("_ppl"):
                to_remove.append(k)
        # del df_modded_runs_group["None_ppl"]
        # del df_budgets["None_ppl"]
    print("TO REMOVE:", to_remove)
    for k in to_remove:
        del df_modded_runs_group[k]
        del df_budgets[k]
    print("REMAINING:", df_modded_runs_group.keys())


    # Remove (ppl) from column names and "None" history rows
    df_modded_runs_group = {k: v.rename(columns=lambda x: x.replace(" (ppl)", "")) for k, v in df_modded_runs_group.items()}
    # df_modded_runs_group = {k: v.drop("None") for k, v in df_modded_runs_group.items()}

    # Do the same for budgets
    df_budgets = {k: v.rename(columns=lambda x: x.replace(" (ppl)", "")) for k, v in df_budgets.items()}
    # df_budgets = {k: v.drop("None") for k, v in df_budgets.items()}

    # Create merged df
    # Keys as a new column, and reset index, history as another new column
    df_merged = pd.concat(df_modded_runs_group, keys=df_modded_runs_group.keys()).reset_index(level=1)
    df_merged_budget = pd.concat(df_budgets, keys=df_budgets.keys()).reset_index(level=1)

    # Reset index again, rename the old one to "Config"
    df_merged = df_merged.reset_index().rename(columns={"index": "Config"})
    df_merged_budget = df_merged_budget.reset_index().rename(columns={"index": "Config"})

    # Add the baseline row at the top
    # TODO: It don't think this is necessary anymore, so I commented it out (May 21, 2023)
    # df_merged = df_merged.append(df_baseline.reset_index().rename(columns={"index": "Config"}))
    # df_merged_budget = df_merged_budget.append(df_baseline_budget.reset_index().rename(columns={"index": "Config"}))

    # Move last row to the top
    df_merged = df_merged.iloc[-1:].append(df_merged.iloc[:-1]).reset_index(drop=True)
    df_merged_budget = df_merged_budget.iloc[-1:].append(df_merged_budget.iloc[:-1]).reset_index(drop=True)

    # Final names for DH
    replacements_2 = {
        # "None": "No DH, no BI",
        "None_1": "No DH, no BI",
        "BART": "BART-D",
        "PegasusFT": "Pegasus-DS",
        "Pegasus-CD": "Pegasus-CD",
    }
    df_merged["History Signal"] = df_merged["History Signal"].replace(replacements_2)
    df_merged_budget["History Signal"] = df_merged_budget["History Signal"].replace(replacements_2)
 
    if target=="ppl":
        replace_dict = {
            "None_ppl": "",
            "PegasusFT_ppl": " + BI(P_FT)",
            "Pegasus-CD_ppl": " + BI"
        }
    elif target=="normal":
        replace_dict = {
            "None_normal": "",
            "PegasusFT_normal": " + BI(P_FT)",
            "Pegasus-CD_normal": " + BI"
        }


    df_merged["Config"] = df_merged["Config"].fillna("").replace(replace_dict)
    df_merged_budget["Config"] = df_merged_budget["Config"].fillna("").replace(replace_dict)

    # Add config to history signal
    df_merged["History Signal"] = df_merged["History Signal"] + df_merged["Config"]
    df_merged_budget["History Signal"] = df_merged_budget["History Signal"] + df_merged_budget["Config"]

    # One "None_0 + BI" might have been added, remove it -- None_0 is results from the orig none_none setup
    df_merged = df_merged[df_merged["History Signal"] != "None_0 + BI"]
    df_merged_budget = df_merged_budget[df_merged_budget["History Signal"] != "None_0 + BI"]

    # Drop all None_0 **
    # df_merged = df_merged[df_merged["History Signal"] != "None_0"]
    # df_merged_budget = df_merged_budget[df_merged_budget["History Signal"] != "None_0"]

    # No DH, No BI + BI -> Only BI
    df_merged["History Signal"] = df_merged["History Signal"].replace({"No DH, no BI + BI": "Only BI"})
    df_merged_budget["History Signal"] = df_merged_budget["History Signal"].replace({"No DH, no BI + BI": "Only BI"})
    
   # df_merged["History Signal"] = df_merged["History Signal"].replace({"None": "No DH, No BI"})
    # df_merged_budget["History Signal"] = df_merged_budget["History Signal"].replace({"None": "No DH, No BI"})

    # make history signal the index column
    df_merged = df_merged.set_index("History Signal")
    df_merged_budget = df_merged_budget.set_index("History Signal")

    # drop config
    df_merged = df_merged.drop("Config", axis=1)
    df_merged_budget = df_merged_budget.drop("Config", axis=1)

    assert (df_merged.index == df_merged_budget.index).all()

    # Convert the column names to tuples by splitting on space
    # df_merged.columns = [tuple(col.split()) for col in df_merged.columns]
    df_merged = df_merged.reindex(framedict['None_normal'].columns, axis=1)
    df_merged_budget = df_merged_budget.reindex(framedict['None_normal'].columns, axis=1)

    # print(df_merged)

    # Fixed model order
    history_order = ["None_0", "No DH, no BI", 
                    #  "Only BI", 
                        "BART-D", "Pegasus-CD", "Pegasus-DS", "Pegasus-DS + BI", 
                        "Recent-1", "Recent-2", "Recent-4", "Recent-8", "Recent-10", "Recent-16", 
                        "Semantic-1", "Semantic-2", "Semantic-4", "Semantic-8", "Semantic-10", "Semantic-16","Full", 
                        ]
    # remove history signals that are not in the data
    history_order_chk = [h for h in history_order if h in df_merged.index]
    df_merged = df_merged.reindex(history_order_chk)
    history_order_chk = [h for h in history_order if h in df_merged_budget.index]
    df_merged_budget = df_merged_budget.reindex(history_order_chk)


    # Plot the merged df using the scatter_plot_pivoted_df function
    f3 = f'plots/{dataset_prefix}/{metric}_merge_{target}.pdf'
    scatter_plot_pivoted_df(df_merged[~df_merged.index.isin(["None_0", "No DH, no BI"])],metric, f3)

    # Get the ROI
    # df_delta = df_merged - df_merged.iloc[0]
    # df_delta_budget = df_merged_budget - df_merged_budget.iloc[0]
    # df_merged_ROI = 10_000 * df_delta / df_delta_budget
    df_merged_ROI = calculate_ROI(df_merged, df_merged_budget)

    # Plot the ROI
    f4 = f'plots/{dataset_prefix}/{metric}_merge_{target}_ROI.pdf'
    scatter_plot_pivoted_df(df_merged[~df_merged.index.isin(["None_0", "No DH, no BI"])],metric, f4, is_roi=True)

    return df_merged, df_merged_budget, df_merged_ROI

def merge_runs_pro(df_merged_normal, df_merged_ppl, metric):
    # Copy the df_merged_normal and df_merged_ppl
    df_merged_normal = df_merged_normal.copy()
    df_merged_ppl = df_merged_ppl.copy()

    # If columns are tuples, convert them to strings
    if isinstance(df_merged_normal.columns[0], tuple):
        df_merged_normal.columns = [" ".join(col) for col in df_merged_normal.columns]
        df_merged_ppl.columns = [" ".join(col) for col in df_merged_ppl.columns]
        
    # Average runs by models+FS+normal/ppl)
    zs_runs_n = [c for c in df_merged_normal.columns if "ZS" in c]
    fs_runs_n = [c for c in df_merged_normal.columns if "FS" in c]
    
    # Create new df
    df_merged = pd.DataFrame()
    df_merged[('Model Avg.','ZS+Manual')] = df_merged_normal[zs_runs_n].mean(axis=1)
    df_merged[('Model Avg.','FS+Manual')] = df_merged_normal[fs_runs_n].mean(axis=1)
    df_merged[('Model Avg.','ZS+PPL')] = df_merged_ppl[zs_runs_n].mean(axis=1)
    df_merged[('Model Avg.','FS+PPL')] = df_merged_ppl[fs_runs_n].mean(axis=1)

    # Plot the merged df using the scatter_plot_pivoted_df function
    f1 = f'plots/{dataset_prefix}/{metric}_merge.avg.pdf'
    scatter_plot_pivoted_df(df_merged[~df_merged.index.isin(["None_0", "No DH, no BI"])],metric, f1)

    return df_merged
 

In [23]:
prompt_type_groups = {
    'normal': ['FS-ICL', 'ZS-ICL'],
    'ppl': ['FS-ICL (ppl)', 'ZS-ICL (ppl)'],
}
metrics = ['BLEU', 'METEOR', 'rougeL', 'Bert-f1', 'BLEURT', 'DEB']

# Start Processing
filtered_df = df[df['History Signal'].notnull()]

# Filter by metric
for metric in metrics:
    all_subframes = {}
    if 'Persona Signal' in df:
        # dataset_prefix = "MSC"
        # Filter by persona
        groups = filtered_df.groupby(['Persona Signal'])

        # Find common baselines
        # Persona = None, History = None, prompt_type=same
        baseline_df = filtered_df[(filtered_df['Persona Signal'] == "None") & (filtered_df['History Signal'] == "None")]
        for signal, sub_df in groups:
            #if signal != "None":
            #    _internal_baseline = sub_df[sub_df['History Signal'] == "None"]
            #    if len(_internal_baseline) == 0:
            #        sub_df_with_baseline = sub_df.append(baseline_df)
            proc_sub_df_persona(sub_df, baseline_df, signal, prompt_type_groups, metric, all_subframes, dataset_prefix)
            # if signal == "None":
            #     raise "Hell"
    else:
        # dataset_prefix = "TC"
        # Find common baselines
        # Persona = None, History = None, prompt_type=same
        baseline_df = filtered_df[(filtered_df['History Signal'] == "None")]

        proc_sub_df_persona(filtered_df, baseline_df, "NA", prompt_type_groups, metric, all_subframes, dataset_prefix)

    # raise Exception("Hell")
    # if dataset_prefix == "MSC":
    if 'Persona Signal' in df:
        df_merged_p, _, df_merged_p_ROI = merge_runs(all_subframes, metric, dataset_prefix, target="ppl")
        df_merged_n, _, df_merged_n_ROI = merge_runs(all_subframes, metric, dataset_prefix, target="normal")
        _ = merge_runs_pro(df_merged_n, df_merged_p, metric)
    # Save all subframes to xlsx, one sheet per persona signal
    # (signal, prompt_types) for all prompt type should be in same sheet for easy comparison

    with pd.ExcelWriter(f'processed/{dataset_prefix}/{metric}.xlsx') as writer:
        for key, value in all_subframes.items():
            # remove multiindex 
            value.columns = [' '.join(col).strip() for col in value.columns.values]
            value.to_excel(writer, sheet_name=key)

        # if dataset_prefix == "MSC":
        if "Persona Signal" in df:
            # Save merged df
            # fix column names
            df_merged_p.columns = [' '.join(col).strip() for col in df_merged_p.columns.values]
            df_merged_p.to_excel(writer, sheet_name="merged_ppl")

            # Save merged ROI
            # fix column names
            df_merged_p_ROI.columns = [' '.join(col).strip() for col in df_merged_p_ROI.columns.values]
            df_merged_p_ROI.to_excel(writer, sheet_name="merged_ppl_ROI")

            # Save merged df
            # fix column names
            df_merged_n.columns = [' '.join(col).strip() for col in df_merged_n.columns.values]
            df_merged_n.to_excel(writer, sheet_name="merged_normal")

            # Save merged ROI
            # fix column names
            df_merged_n_ROI.columns = [' '.join(col).strip() for col in df_merged_n_ROI.columns.values]
            df_merged_n_ROI.to_excel(writer, sheet_name="merged_normal_ROI")



/home/bytestorm/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-21-64776c2fad98>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompt_baseline_df['History Signal'] = "None_0"


History: ['Full' 'PegasusFT' 'BART' 'Recent-2' 'Recent-4' 'Recent-8' 'Recent-10'
 'Semantic-2' 'Semantic-4' 'Semantic-8' 'Semantic-10' 'None_1'
 'Pegasus-CD' 'Recent-1' 'Semantic-1' 'None_0']
Prompt types: ['FS-ICL' 'ZS-ICL']
Metric: BLEU
Save path: plots/TC/BLEU_None_normal.pdf


/home/bytestorm/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-21-64776c2fad98>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompt_baseline_df['History Signal'] = "None_0"


History: ['Full' 'PegasusFT' 'BART' 'Recent-2' 'Recent-4' 'Recent-8' 'Recent-10'
 'Semantic-2' 'Semantic-4' 'Semantic-8' 'Semantic-10' 'None_1'
 'Pegasus-CD' 'Recent-1' 'Semantic-1' 'None_0']
Prompt types: ['FS-ICL (ppl)' 'ZS-ICL (ppl)']
Metric: BLEU
Save path: plots/TC/BLEU_None_ppl.pdf


/home/bytestorm/miniconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-21-64776c2fad98>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompt_baseline_df['History Signal'] = "None_0"


...

History: ['PegasusFT' 'BART' 'None_0']
Prompt types: ['FS-ICL (ppl)' 'ZS-ICL (ppl)']
Metric: DEB
Save path: plots/TC/DEB_Pegasus-CD_ppl.pdf
TO REMOVE: ['None_normal', 'Pegasus-CD_normal']
REMAINING: dict_keys(['None_ppl', 'Pegasus-CD_ppl'])
Metric: DEB
Save path: plots/TC/DEB_merge_ppl.pdf


/home/bytestorm/miniconda3/lib/python3.9/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Metric: DEB
Save path: plots/TC/DEB_merge_ppl_ROI.pdf
TO REMOVE: ['None_ppl', 'Pegasus-CD_ppl']
REMAINING: dict_keys(['None_normal', 'Pegasus-CD_normal'])
Metric: DEB
Save path: plots/TC/DEB_merge_normal.pdf


/home/bytestorm/miniconda3/lib/python3.9/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Metric: DEB
Save path: plots/TC/DEB_merge_normal_ROI.pdf
Metric: DEB
Save path: plots/TC/DEB_merge.avg.pdf


In [24]:
merge_runs_pro(df_merged_n, df_merged_p, metric)

Metric: DEB
Save path: plots/TC/DEB_merge.avg.pdf


,"(Model Avg., ZS+Manual)","(Model Avg., FS+Manual)","(Model Avg., ZS+PPL)","(Model Avg., FS+PPL)"
History Signal,,,,
None_0,0.876117,0.949751,0.876117,0.949751
"No DH, no BI",0.876117,0.949751,0.804145,0.891926
BART-D,0.836667,0.750229,0.813121,0.663412
Pegasus-CD,0.890246,0.758013,0.861943,0.595830
Pegasus-DS,0.838892,0.750983,0.814472,0.667482
Pegasus-DS + BI,0.857621,0.730233,0.850182,0.683912
Recent-1,0.865879,0.812345,0.811634,0.851828
Recent-2,0.878496,0.811646,0.849782,0.867925
Recent-4,0.885858,0.805802,0.868605,0.878070


In [25]:
df_merged_n

,flanT5-XL FS-ICL,flanT5-XL ZS-ICL,T0 FS-ICL,T0 ZS-ICL,Tk-Instruct FS-ICL,Tk-Instruct ZS-ICL,GPT-3 FS-ICL,GPT-3 ZS-ICL
History Signal,,,,,,,,
None_0,0.958204,0.871732,0.968648,0.935264,0.950834,0.837493,0.921320,0.859980
"No DH, no BI",0.958204,0.871732,0.968648,0.935264,0.950834,0.837493,0.921320,0.859980
BART-D,0.748232,0.770405,0.708383,0.922652,0.721551,0.786473,0.822749,0.867140
Pegasus-CD,0.823246,0.835588,0.597069,0.924177,0.700447,0.894951,0.911289,0.906269
Pegasus-DS,0.751057,0.775066,0.705942,0.923344,0.722671,0.790558,0.824261,0.866598
Pegasus-DS + BI,0.771571,0.830483,0.640035,0.931706,0.701979,0.799220,0.807347,0.869076
Recent-1,0.567848,0.822037,0.920688,0.918384,0.876031,0.863552,0.884813,0.859543
Recent-2,0.568873,0.826602,0.919616,0.920718,0.862600,0.888921,0.895496,0.877742
Recent-4,0.567367,0.826899,0.919493,0.921281,0.832034,0.905339,0.904315,0.889914


In [26]:
df_merged_p

,flanT5-XL FS-ICL,flanT5-XL ZS-ICL,T0 FS-ICL,T0 ZS-ICL,Tk-Instruct FS-ICL,Tk-Instruct ZS-ICL,GPT-3 FS-ICL,GPT-3 ZS-ICL
History Signal,,,,,,,,
None_0,0.958204,0.871732,0.968648,0.935264,0.950834,0.837493,0.92132,0.85998
"No DH, no BI",0.835805,0.745327,0.947230,0.844509,0.892743,0.822598,NaN,NaN
BART-D,0.733770,0.768278,0.485161,0.870441,0.771304,0.800645,NaN,NaN
Pegasus-CD,0.802168,0.841282,0.333552,0.889625,0.651769,0.854922,NaN,NaN
Pegasus-DS,0.739330,0.770427,0.494839,0.871279,0.768276,0.801710,NaN,NaN
Pegasus-DS + BI,0.759812,0.792150,0.628158,0.914411,0.663767,0.843986,NaN,NaN
Recent-1,0.815727,0.783195,0.889686,0.874325,0.850071,0.777381,NaN,NaN
Recent-2,0.834819,0.821352,0.893178,0.889057,0.875777,0.838937,NaN,NaN
Recent-4,0.847949,0.834340,0.895526,0.898493,0.890736,0.872981,NaN,NaN


In [27]:
all_subframes["Pegasus-CD_ppl"]

,flanT5-XL FS-ICL (ppl),flanT5-XL ZS-ICL (ppl),T0 FS-ICL (ppl),T0 ZS-ICL (ppl),Tk-Instruct FS-ICL (ppl),Tk-Instruct ZS-ICL (ppl),GPT-3 FS-ICL (ppl),GPT-3 ZS-ICL (ppl)
History Signal,,,,,,,,
None_0,0.958204,0.871732,0.968648,0.935264,0.950834,0.837493,0.92132,0.85998
BART,0.759812,0.792150,0.628158,0.914411,0.663767,0.843986,NaN,NaN
PegasusFT,0.759812,0.792150,0.628158,0.914411,0.663767,0.843986,NaN,NaN


In [28]:
import pandas as pd

# assuming all_subframes is a dictionary containing the subframes
none_normal = all_subframes['None_normal'].loc[['None_1']]
none_ppl = all_subframes['None_ppl'].loc[['None_1']]

# display the subframes side by side
pd.concat([none_normal, none_ppl], axis=0)

,flanT5-XL FS-ICL,flanT5-XL ZS-ICL,T0 FS-ICL,T0 ZS-ICL,Tk-Instruct FS-ICL,Tk-Instruct ZS-ICL,GPT-3 FS-ICL,GPT-3 ZS-ICL,flanT5-XL FS-ICL (ppl),flanT5-XL ZS-ICL (ppl),T0 FS-ICL (ppl),T0 ZS-ICL (ppl),Tk-Instruct FS-ICL (ppl),Tk-Instruct ZS-ICL (ppl),GPT-3 FS-ICL (ppl),GPT-3 ZS-ICL (ppl)
History Signal,,,,,,,,,,,,,,,,
None_1,0.958204,0.871732,0.968648,0.935264,0.950834,0.837493,0.92132,0.85998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
None_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.835805,0.745327,0.94723,0.844509,0.892743,0.822598,NaN,NaN


In [29]:
sub_df[["History Signal", "Model", "Prompt Type"]].value_counts()

History Signal  Model        Prompt Type 
BART            GPT-3        FS-ICL          1
                             ZS-ICL          1
PegasusFT       flanT5-XL    ZS-ICL          1
                             FS-ICL (ppl)    1
                             FS-ICL          1
                Tk-Instruct  ZS-ICL (ppl)    1
                             ZS-ICL          1
                             FS-ICL (ppl)    1
                             FS-ICL          1
                T0           ZS-ICL (ppl)    1
                             ZS-ICL          1
                             FS-ICL (ppl)    1
                             FS-ICL          1
                GPT-3        ZS-ICL          1
                             FS-ICL          1
BART            flanT5-XL    ZS-ICL (ppl)    1
                             ZS-ICL          1
                             FS-ICL (ppl)    1
                             FS-ICL          1
                Tk-Instruct  ZS-ICL (ppl)    1
                  

In [30]:
signal

'Pegasus-CD'

In [31]:
%%bash 
    #!/bin/bash
    # if figlet is installed
    if [ -x "$(command -v figlet)" ]; then
        figlet "Adios!"
    fi

    _       _ _           _ 
   / \   __| (_) ___  ___| |
  / _ \ / _` | |/ _ \/ __| |
 / ___ \ (_| | | (_) \__ \_|
/_/   \_\__,_|_|\___/|___(_)
                            
